In [1]:
%%capture --no-stderr
%pip install -quit -U langchain_openai langchain_core langgraph langgraph-supervisor

In [2]:
import os, getpass

def _set_env(var: str):
        if not os.environ.get(var):
            os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [3]:
import datetime

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o")

def current_date_time() -> str:
    "This will give you the current date and time in your location. And maybe the day of the week, idk"
    
    dt = datetime.datetime.now()
    return "Datetime is {} and today is {}".format(dt, dt.strftime('%A'))

time_keeper_agent = create_react_agent(
    model=llm,
    tools=[current_date_time],
    name = "time_keeper_agent",
    prompt= "You tell the current date, time and day of the week in your location."
)

In [4]:
def shop_opening_hours() -> str:
    "The shop's opening hours are:"
    schedule = """Monday: 10am-8pm
                    Tuesday: 10am-8pm
                    Wednesday: 10am-8pm
                    Thursday: 10am-8pm
                    Friday: 10am-8pm
                    Saturday: 10am-8pm
                    Sunday: 10am-8pm"""
    return schedule

schedule_keeper_agent = create_react_agent(
    model=llm,
    tools=[shop_opening_hours],
    name = "schedule_keeper_agent",
    prompt= "You tell the shop's opening hours."
)

In [5]:
from langgraph_supervisor import create_supervisor

prompt = ("""You are a team supervisor to manage time_keeper_agent and schedule_keeper_agent.
            Your job is to respond to shop opening hour queries using the time_keeper_agent and schedule_keeper_agent.
            To get the current time and date, use time)_keeper_agent and to get the shop's operating hours
            use the the schedule_keeper_agent. You can use both agents in parallel to get the information.""")

workflow = create_supervisor(
    [schedule_keeper_agent, time_keeper_agent],
    model=llm,
    output_mode = "full_history",
    prompt = prompt
)

ModuleNotFoundError: No module named 'langgraph_supervisor'